In [1]:
# Dependencies
from bs4 import BeautifulSoup
from splinter import Browser
import requests
import pymongo
import pandas as pd

# Import connection strings
from config import host_name
from config import client_name

In [13]:
# chromedriver path
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Initialize PyMongo to work with MongoDBs
# conn = f'mongodb://{host_name}'
# client = f'pymongo.{client_name}(conn)'

In [5]:
# Define database
# db = client.mars_db

## Step 1 - Data Scrapping

### NASA Mars News

In [46]:
# Scrape the latest Mars news 
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url)

In [47]:
# Create BeautifulSoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [67]:
# Find the title of the latest article
news_title = soup.find_all('div', class_='bottom_gradient')[0].get_text()
news_title

"NASA Engineers Checking InSight's Weather Sensors"

In [68]:
 # Find the teaser of the latest article
news_p = soup.find_all('div', class_='article_teaser_body')[0].get_text()
news_p

'An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.'

### JPL Mars Space Images

In [39]:
# Scrape featured image from webpage
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [40]:
# Create BeautifulSoup object
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [44]:
# Find and extract the current featured image
img = soup.find('div', class_='carousel_items').article['style']
img

"background-image: url('/spaceimages/images/wallpaper/PIA14762-1920x1200.jpg');"

In [45]:
# Extract the endpoints to the image url
new_img = img.split("'")[1]
new_img

'/spaceimages/images/wallpaper/PIA14762-1920x1200.jpg'

In [42]:
# Create url of the image
featured_image_url = 'https://www.jpl.nasa.gov' + new_img
featured_image_url

'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA14762-1920x1200.jpg'

### Mars Facts

In [10]:
# Scrape tabular data from webpage
url = 'https://space-facts.com/mars/'
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:     -87 to -5 °C      -88 to 58°C,
           

In [11]:
# Convert table into dataframe
df = tables[0]
df.head()

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"


In [12]:
# Read dataframe into html and export codes to file
df.to_html("output.html", index=False, header=False)

### Mars Hemispheres

## Step 2 - MongoDB & Flask App